In [21]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GroupKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

## Custom libraries
import index_helpers as ih
from data_transformations import transformation_call
from data_transformations import low_var_exclusion

In [22]:
#choice of segmentation
segmentation = True
fine_segmentation=True

#Try Expert, Metadata, Normalization, Power
exclude_expert=False
exclude_meta_data=False
power=False
normalization=False
treshold=13
## Import, index, and split
df = ih.read_and_merge_data(segmentation, fine_segmentation, exclude_expert, exclude_meta_data)
df = transformation_call(df, normalization, power)
df = ih.index_df_by_person(df)
df = low_var_exclusion(df, treshold)
df = ih.categorical_float_to_int(df)
df = ih.categorical_to_dummy(df)

X_train, X_val, y_train, y_val = ih.train_test_split_on_index(features = df.drop('Label', axis=1),
                                                             label = df["Label"])

## Modify data for GroupKFold
groups = y_train.reset_index()['File_Name_split']
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)


In [23]:
#Linear Discriminant Analysis

## Make pipeline - name classifier "clf"
clf_pipeline = Pipeline([("st_scaler", StandardScaler()),
                        ("clf", LinearDiscriminantAnalysis())])

## Use "clf__" in order to correctly assign parameters to the clf object
clf_param_grid = {'clf__tol':[0.0003, 0.0002, 0.0001, 0.00005],
                 'clf__solver':['svd', 'lsqr','eigen']}

## Instantiate GroupKFold to avoid data leakage - to be passed to cv
gkf=GroupKFold(n_splits=10)

## Set up Randomized search CV
clf_rand_auc = RandomizedSearchCV(estimator=clf_pipeline,
                                  param_distributions=clf_param_grid,
                                  cv=gkf, scoring='roc_auc', verbose=1, n_jobs=2, n_iter=50)

## Perform Group K-Cross-validation
clf_rand_auc.fit(X_train, y_train, groups=groups)

## Print results
print("Best score: ",  clf_rand_auc.best_score_)
print("Best estimator: ", clf_rand_auc.best_estimator_)

C:\Users\chris\anaconda\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 12 is smaller than n_iter=50. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 10 folds for each of 12 candidates, totalling 120 fits
Best score:  0.6941258357188721
Best estimator:  Pipeline(steps=[('st_scaler', StandardScaler()),
                ('clf', LinearDiscriminantAnalysis(tol=0.0003))])


[Parallel(n_jobs=2)]: Done 120 out of 120 | elapsed:    2.9s finished


In [24]:
from sklearn.metrics import roc_auc_score
pred=clf_rand_auc.predict(X_val)
roc_auc_score(pred,y_val)

0.7297275340582428